In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy import ndarray

In [ ]:
def apply_medfilt_grayscale(noisy_img:np.ndarray, filter_size:int, label:str) -> tuple:
    """
    This filter is a non-linear filter that replaces each pixel with the median of its neighboring pixels.
    The median filter is an effective way to remove noise from an image while preserving edges.
    The median filter can be applied with a probability by randomly choosing which pixels to filter and which to leave unchanged.
    """
    if filter_size == None:
        return noisy_img, noisy_img

    filtered_img = cv2.medianBlur(noisy_img, ksize=filter_size)
    fig = plt.figure()

    plt.imshow(filtered_img, cmap=plt.cm.gray)
    plt.title(f"filter size:{filter_size} median filtered {label}", fontsize=12)

    fig.tight_layout(pad=2.0)
    plt.show()
    return noisy_img, filtered_img

In [ ]:
def add_uniform_noise(img_path:str, probability) -> []:
    original_image = cv2.imread(f"{img_path}")
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    Uniform_Noise = np.zeros_like(original_image)

    cv2.randu(Uniform_Noise, 0, 255)

    uni_noise = (Uniform_Noise * 1.0).astype(np.uint8)
    mask = np.random.choice([0, 1], size=uni_noise.shape, p=[1-probability, probability]).astype(np.uint8)
    uni_noise = (uni_noise * mask).astype(np.uint8)

    uni_noised_img = cv2.add(original_image, uni_noise)

    fig = plt.figure(dpi=128)
    fig.add_subplot(1,1,1)
    plt.imshow(uni_noised_img)
    plt.title(f"p:{probability} Uniform Noised {img_path}", fontsize=10)

    fig.tight_layout(pad=1.0)
    plt.imshow(uni_noised_img,cmap=plt.cm.gray)
    plt.show()
    print(f"uniform noised {img_path} with probability : {probability}")
    return original_image, uni_noised_img

In [ ]:
uniform_noise_applied_images ={f"{p}_{img_p}": add_uniform_noise(img_p, p)[1] for img_p in ["m_high.jpg", "m_low.jpg", "m_mid.jpg"] for p in [0.0, 0.1, 0.5, 0.8]}

In [ ]:
filter_sizes=[None, 5, 15]
filtered_noisy_images = {f"{f_size}x{f_size}_filtered_{p}_{img_p}" :  apply_medfilt_grayscale(uniform_noise_applied_images[f"{p}_{img_p}"], f_size, f"{img_p}")[1] for p in [0.0, 0.1, 0.5, 0.8] for img_p in ["m_high.jpg", "m_mid.jpg", "m_low.jpg"]
                         for f_size in filter_sizes}


In [ ]:
len(filtered_noisy_images.keys())

In [ ]:
filtered_noisy_images.keys()

In [ ]:
plt.imshow(filtered_noisy_images["NonexNone_filtered_0.0_m_high.jpg"], cmap=plt.cm.gray)


In [ ]:
plt.imshow(filtered_noisy_images["NonexNone_filtered_0.8_m_high.jpg"], cmap=plt.cm.gray)


In [ ]:
plt.imshow(filtered_noisy_images["5x5_filtered_0.8_m_high.jpg"], cmap=plt.cm.gray)


In [ ]:
plt.imshow(filtered_noisy_images["15x15_filtered_0.8_m_high.jpg"], cmap=plt.cm.gray)


# 1) clustering based segmentation

In [ ]:
def apply_clustering_segmentation(image):
    image = np.float32(image)

    # Reshape image for clustering (rows*cols, channels)
    reshaped_image = image.reshape((-1, 1))

    # Perform K-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(reshaped_image, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Reshape labels to match original image shape
    segmented_image = labels.reshape(image.shape)

    return segmented_image

# 2) split-merge segmentation

In [ ]:
def apply_split_merge_segmentation(image):
    # Apply threshold to create a binary image
    _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Apply the watershed algorithm for split-merge segmentation
    _, markers = cv2.connectedComponents(thresholded_image)
    segmented_image = cv2.watershed(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), markers)

    return segmented_image

# 3) auto-threshold segmentation

In [ ]:
def apply_auto_threshold_segmentation(image):
    # Apply Otsu's thresholding for segmentation
    _, segmented_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return segmented_image

In [ ]:
segmentation_methods = [("clustering", apply_clustering_segmentation), ("split_merge", apply_split_merge_segmentation),
                        ("auto", apply_auto_threshold_segmentation)]


In [ ]:
for sm in segmentation_methods:
    print(f"{sm[0]} will use function {sm[1]}")

In [ ]:
plt.imshow(apply_clustering_segmentation(filtered_noisy_images["NonexNone_filtered_0.0_m_low.jpg"]))

In [ ]:
import time
start=time.time()
segmented_filtered_noised_images = {f"{segment_method[0]}_segmented_{f_size}x{f_size}_filtered_{p}_noised_{img_p}": segment_method[1](filtered_noisy_images[f"{f_size}x{f_size}_filtered_{p}_{img_p}"]) for segment_method in segmentation_methods for f_size in filter_sizes for p in [0.0, 0.1, 0.5, 0.8] for img_p in ["m_high.jpg", "m_mid.jpg", "m_low.jpg"]}
end=time.time()
print(f"{end-start:.2f} seconds to finish process.")

In [ ]:
plt.imshow(segmented_filtered_noised_images[f"clustering_segmented_5x5_filtered_0.0_noised_m_low.jpg"])

In [ ]:
plt.imshow(segmented_filtered_noised_images[f"auto_segmented_5x5_filtered_0.0_noised_m_low.jpg"])

In [ ]:
plt.imshow(segmented_filtered_noised_images[f"split_merge_segmented_NonexNone_filtered_0.0_noised_m_high.jpg"])

# Results

In [ ]:
image_set = ["m_high.jpg", "m_mid.jpg", "m_low.jpg"]
probabilities = [0.0, 0.1, 0.5, 0.8]
f_sizes = filter_sizes
segmentations = segmentation_methods

In [ ]:
results = segmented_filtered_noised_images.copy()

In [ ]:
results.keys()

In [ ]:
for method_index in [0,1,2]:
    for  filter_index in [0,1,2]:
        fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(24,13.5))

        f_size = f_sizes[filter_index]
        segmentation = segmentations[method_index]

        fig.suptitle(f"Segmentation method: {segmentation[0]}, EPF size: {f_size}", fontsize=24)
        for i, img in enumerate(image_set):
            for j, proba in enumerate(probabilities):
                ax[i, j].imshow(results[f"{segmentation[0]}_segmented_{f_size}x{f_size}_filtered_{proba}_noised_{img}"])
                ax[i, j].set_title(f"{img}, p:{proba}", fontsize=14)

        plt.savefig(f"segmentation_{segmentation[0]}_epfsize_{f_size}_images.jpg")
        plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(24,13.5))

index = 1
f_size = f_sizes[index]
segmentation = segmentations[index]

fig.suptitle(f"Segmentation method: {segmentation[0]}, EPF size: {f_size}", fontsize=24)
for i, img in enumerate(image_set):
    for j, proba in enumerate(probabilities):
        ax[i, j].imshow(results[f"{segmentation[0]}_segmented_{f_size}x{f_size}_filtered_{proba}_noised_{img}"])
        ax[i, j].set_title(f"{img}, p:{proba}", fontsize=14)

plt.savefig(f"segmentation_{segmentation[0]}_epfsize_{f_size}_images.jpg")
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(24,13.5))

index = 2
f_size = f_sizes[index]
segmentation = segmentations[index]

fig.suptitle(f"Segmentation method: {segmentation[0]}, EPF size: {f_size}", fontsize=24)
for i, img in enumerate(image_set):
    for j, proba in enumerate(probabilities):
        ax[i, j].imshow(results[f"{segmentation[0]}_segmented_{f_size}x{f_size}_filtered_{proba}_noised_{img}"])
        ax[i, j].set_title(f"{img}, p:{proba}", fontsize=14)

plt.savefig(f"segmentation_{segmentation[0]}_epfsize_{f_size}_images.jpg")
plt.show()